In [13]:
using DataFrames
using DataFramesMeta
using Dates

using PowerSystems
using PowerSimulations
using StorageSystemsSimulations
using HydroPowerSimulations
using PowerAnalytics
using PowerAnalytics.Metrics
using PowerAnalytics.Selectors
using PowerGraphics
using Gurobi
using HiGHS

In [14]:
system = System("./models/sienna/miso_base.json")

┌ Info: Loaded time series from storage file existing=./models/sienna\miso_base_time_series_storage.h5 new=C:\Users\UEU112~1\AppData\Local\Temp\jl_C66E.tmp compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems C:\Users\UEU112259C\.julia\packages\InfrastructureSystems\LEg3t\src\hdf5_time_series_storage.jl:110


Property,Value
Name,
Description,
System Units Base,DEVICE_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,1194
Type,Count
ACBus,3
Arc,2
Area,3


In [ ]:
template = ProblemTemplate()

set_network_model!(
    template,
    NetworkModel(CopperPlatePowerModel, use_slacks = true)
)

set_device_model!(template, PowerLoad, StaticPowerLoad)

set_device_model!(template, ThermalStandard, ThermalBasicDispatch)
set_device_model!(template, RenewableDispatch, RenewableFullDispatch)
set_device_model!(template, HydroDispatch, HydroDispatchRunOfRiver)
set_device_model!(
    template,
    DeviceModel(
        EnergyReservoirStorage,
        StorageDispatchWithReserves;
        attributes=Dict(
            "reservation" => false,
            "energy_target" => false,
            "cycling_limits" => false,
            "regularization" => false,
        ),
    )
)

# solver = optimizer_with_attributes(Gurobi.Optimizer, "MIPGap" => 0.5) 
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5)
problem = DecisionModel(
    template,
    system;  
    optimizer = solver,
    horizon = Hour(8760),
    resolution = Hour(1),
    optimizer_solve_log_print = true,
    calculate_conflict = true,
    store_variable_names = true,
)
build!(problem; output_dir = mktempdir())

solve!(problem)

results = OptimizationProblemResults(problem)

In [ ]:
computations = [
    (calc_load_forecast, all_loads, "load"),    
    (calc_active_power, make_selector(ThermalStandard; groupby = :all), "thermal_power"),
    # (calc_active_power, make_selector(HydroDispatch; groupby = :all), "hydro_power"),
    (calc_active_power, make_selector(RenewableDispatch; groupby = :all), "renewable_power"),
    (calc_curtailment, make_selector(RenewableDispatch; groupby = :all), "renewable_curtailment"),
]

# compute_all(results_base, computations...)
compute_all(results_no_min_gen, computations...)

In [ ]:
# plotlyjs()
# pal = load_palette("$basepath/misc/color-palette-2.yaml")
# ;

In [ ]:
# plot_fuel(results_base, generator_mapping_file="$basepath/misc/generator_mapping.yaml", curtailment=false)